# **Microplane MS1 - uncoupled N-T interface**

This sheets shows the application of the microplane integration
on an example of VUNTIM (vectorized implementation of the uncoupled NT Interface Model.

In [1]:
%matplotlib widget
from bmcs_matmod.msx import MSX
import matplotlib.pylab as plt
import numpy as np
np.seterr(divide='ignore', invalid='ignore');
from bmcs_matmod.msx.energy_dissipation import EnergyDissipation
from ibvpy.tfunction import TimeFunction, TFSelector, TFCyclicSin, TFCyclicNonsymmetricConstant
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.api import XDomainSinglePoint, MATS3DScalarDamage, TStepBC, BCDof
from bmcs_matmod.ms1 import MS12D
from ibvpy.tmodel.mats3D import MATS3DElastic

The example below demonstrate a case with ideally plastic response which delivers
several peaks in the response. At the same time, it can be used to demonstrate that 
$\nu \leq 0.25$ leads does not deliver any solution upon return mapping due to a square root overflow.

Once $E_\mathrm{T}$ becomes negative, the return mapping algorithm cannot work properly.
Is there some thermodynamically admissible interpretation of this case in the interface?
This can be posed in two questions:
 - is it posssible to obtain a positive dissipation of an interface even for negative
   value of $E_\mathrm{T}$?
 - if yes, is it allowed to reach an inelastic domain in the tangential direction?

In [70]:
# E = 39000
# nu = 0.2
# mp = dict(          
# Ad = 700,
# eps_0  = 0.00001,
# sig_0  = 50,
# K_N = 0,
# gamma_N = 80000,
# sigma_T_0  = 10.0,
# K_T  = 0,
# gamma_T  = 5000,
# S_T  = 0.01,
# c_T  = 10,
# r_T  = 13,
# a  = 0.5,
# e_T  = 6.0)

# E = 39226
# nu = 0.18
# mp = dict(          
# Ad = 7000,
# eps_0  = 0.00001,
# sig_0  = 40,
# K_N = 10000,
# gamma_N = 60000,
# sigma_T_0  = 2.0,
# K_T  = 0,
# gamma_T  = 5000,
# S_T  = 0.003,
# c_T  = 10,
# r_T  = 10,
# a  = 0.1,
# e_T  = 6.0)

E = 39226
nu = 0.18
mp = dict(          
Ad = 1800,
eps_0  = 0.00001,
sig_0  = 45,
K_N = 17000,
gamma_N = 9000,
sigma_T_0  = 2.0,
K_T  =20000,
gamma_T  = 500000,
S_T  = 0.0075,
c_T  = 9,
r_T  = 15,
a  = 0.004,
e_T  = 10.0)

ERROR! Session/line number was not unique in database. History logging moved to new session 2261


In [71]:
%matplotlib widget
mpl = MSX()
m_elastic = MATS3DElastic()
mpl.trait_set(E=E, nu=nu, 
          mic='untim', eps_max=0.01, n_eps=30, double_pvw=True)
mpl.mic_.trait_set(**mp)
mpl.mic_.eps_max = 0.02
# mpl.interact()
# m13 = MS13D(**CAxisymm)

In [72]:
tf_precrompression = TFBilinear(loading_ratio = 1.0, time_ratio=0.1)
tf_sliding = TFBilinear(loading_ratio = 0.0, time_ratio=0.1)

In [73]:
mpl.mic_

In [83]:
compression_level = np.array([0,-15,-30,-45])
# compression_level = np.array([0])

m_list_monotonic = []

for compression in compression_level:

    xmodel = XDomainSinglePoint()
    m = TStepBC(
        domains=[(xmodel, mpl),],
        bc=[BCDof(
                var='u', dof=3, value=-0.015,
             time_function=tf_sliding) , BCDof(
                var='f', dof=1, value=compression,
             time_function=tf_precrompression),
           BCDof(
                var='f', dof=2, value=0,
             time_function=tf_precrompression)]
    )
    # m = TStepBC(
    #     domains=[(xmodel, mpl),],
    #     bc=[BCDof(
    #             var='u', dof=0, value=-0.01,
    #          time_function=tf_sliding)]
    # )


    m.sim.tline.trait_set(step=0.005)
    m.sim.reset()
    try:
        m.sim.run()
    except:
        pass
    m_list_monotonic.append(m)

In [84]:
%matplotlib widget
import matplotlib.pylab as plt
fig, ((ax1,ax2)) = plt.subplots(1, 2)

max_F = []
for m in m_list_monotonic:
    ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,3])
    # ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,3])
    max_F.append(min(m.hist.F_t[:,3]))

    # ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,1])
    # ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,2])
    # ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,4])'
    # ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,5])
    # ax2.plot(m.hist.U_t[:,3], -m.hist.U_t[:,1] / m.hist.U_t[:,0])
    ax2.plot(m.hist.U_t[:,3], m.hist.F_t[:,1])
max_F

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[-20.826758258431134,
 -22.07338773098975,
 -28.915383405699888,
 -28.09740581911253]

In [76]:
for m in m_list_monotonic:
    energydissipation = EnergyDissipation()
    energydissipation.plot_energy_dissp(m, mpl)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Fatigue

In [77]:
number_of_cycles = 5
shift_cycles = 1
steps_branch = 25
tf_cyclic = TFCyclicNonsymmetricConstant(number_of_cycles=number_of_cycles, unloading_ratio=0.05, shift_cycles=shift_cycles)
tf_confinement = TFBilinear(loading_ratio=1, time_ratio=1/(number_of_cycles+shift_cycles))

In [78]:
max_F

[-20.826758258431134,
 -34.23942928655724,
 -42.70607895364319,
 -39.47771629162216]

In [79]:
compression_level = np.array([0,-15,-30, -45])

m_list_cyclic = []

for compression, max_shear in zip(compression_level, max_F):

    xmodel = XDomainSinglePoint()
    m = TStepBC(
        domains=[(xmodel, mpl),],
        bc=[BCDof(
                var='f', dof=3, value= max_shear * 0.85,
             time_function= tf_cyclic) , BCDof(
                var='f', dof=1, value=compression,
             time_function=tf_confinement), BCDof(
                var='f', dof=2, value=compression,
             time_function=tf_confinement)]
    )


    m.sim.tline.trait_set(step=1/(steps_branch*2*(number_of_cycles+1)))
    m.sim.reset()
    # m.sim.run()
    try:
        m.sim.run()
    except:
        pass
    m_list_cyclic.append(m)

In [80]:
%matplotlib widget
import matplotlib.pylab as plt
fig, ((ax1,ax2)) = plt.subplots(1, 2)


for m in m_list_cyclic:
    # ax1.plot(m.hist.U_t[:,0], m.hist.F_t[:,0]*2*np.pi*29*30/1000)
    ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,3])
    print(xmodel.map_U_to_field(m.hist.F_t[-1]))

    # ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,1])
    # ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,2])
    # ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,4])
    # ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,5])
    # ax2.plot(m.hist.U_t[:,3], -m.hist.U_t[:,1] / m.hist.U_t[:,0])
    ax2.plot(m.hist.U_t[:,3], m.hist.F_t[:,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[[ 7.02260516e-08  2.94071897e-16 -3.65625634e-17]
  [ 2.94071897e-16  2.85447935e-07 -8.51501837e+00]
  [-3.65625634e-17 -8.51501837e+00  2.85447943e-07]]]
[[[ 2.17887278e-06  1.42094615e-16  8.49112604e-18]
  [ 1.42094615e-16 -1.49999839e+01 -7.27580551e-01]
  [ 8.49112604e-18 -7.27580551e-01 -1.49999839e+01]]]
[[[ 6.06691123e-06  2.37692583e-16  3.09107534e-17]
  [ 2.37692583e-16 -2.99999896e+01 -9.07504817e-01]
  [ 3.09107534e-17 -9.07504817e-01 -2.99999896e+01]]]
[[[ 9.97872270e-06 -2.23357816e-16  7.44022512e-18]
  [-2.23357816e-16 -4.49999718e+01 -8.38907017e-01]
  [ 7.44022512e-18 -8.38907017e-01 -4.49999718e+01]]]


In [ ]:
cm = 1/2.54  # centimeters in inches
plt.style.use('grayscale')
N=[]
plt.tight_layout()
plt.rcParams["font.family"] = "Times New Roman"

fig, ((ax,ax2,ax3)) = plt.subplots(1, 3,figsize=(25*cm, 12*cm))

start = int((shift_cycles+2)*steps_branch)
end = int(steps_branch)
plt.tight_layout()

for m,compression in zip(m_list_cyclic,compression_level):
    
    U_slide = m.hist.U_t[:,3]
    cycles = np.linspace(0, len(U_slide[start::steps_branch*2]),len(U_slide[start:-steps_branch:steps_branch*2]))
    U_dilatancy = m.hist.U_t[:,0]
    F_slide = m.hist.F_t[:,3]
    F_confinement = m.hist.F_t[:,0]
    print(cycles)

    N.append(cycles)
    # ax1.plot(cycles, F_confinement[start:-steps_branch:steps_branch*2], label=str(compression))
    # ax1.plot(U_slide, F_confinement, label=str(compression))
    ax.set_xlabel('confinement [MPa]')
    ax.set_ylabel('cycle [-]')
    # ax1.set_xlim(-5,55)

    ax2.plot(U_slide, F_slide, label=str(compression))
    ax2.set_ylabel('shear force [MPa]')
    ax2.set_xlabel('shear strain [-]')
        # ax2.set_ylim(-1,130)
        # ax2.set_xlim(-0.0005,0.02)

    ax3.plot(cycles, abs(U_slide[start:-end:steps_branch*2]), label=str(compression))
    # ax3.plot(cycles[1:], U_slide[start+steps_branch:-end:steps_branch*2], label=str(compression))
    ax3.set_ylabel('fatigue creep [-]')
    ax3.set_xlabel('cycle [-]')
    # ax3.set_ylim(-0.0005,0.03)
    
ax.plot(compression_level, N, '--ks', markersize=4)
ax.set_xlim(ax.get_xlim()[::-1])
ax.set_yscale('log')
ax.set_ylim(1e0,1e4)

left  = 0.1  # the left side of the subplots of the figure
right = 0.98  # the right side of the subplots of the figure
bottom = 0.2   # the bottom of the subplots of the figure
top = 0.9      # the top of the subplots of the figure
wspace = 0.4   # the amount of width reserved for blank space between subplots
hspace = 0.4   # the amount of height reserved for white space between subplots
# mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["k", "r", "b"]) 
plt.subplots_adjust(left=left, bottom=bottom, right=right, top=None, wspace=wspace, hspace=hspace)

txt=str(CAxisymm)
plt.figtext(0.5, 0.01, txt, wrap=True, horizontalalignment='center', fontsize=10)
# ax1.set_xlim(-1,21)
ax1.set_ylim(-32,2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[0.]
[0.   1.25 2.5  3.75 5.  ]
[0.   1.25 2.5  3.75 5.  ]
[0.   1.25 2.5  3.75 5.  ]


C:\Users\marag\miniconda3\envs\bmcs_env\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [871]:
sigma_ab, eps_ab = [], []
for i in range(len(m.hist.F_t)):
    sigma_ab.append(m.fe_domain[0].xmodel.map_U_to_field(m.hist.F_t[i]))
    eps_ab.append(m.fe_domain[0].xmodel.map_U_to_field(m.hist.U_t[i]))
sigma_ab = np.array(sigma_ab)
eps_ab = np.array(eps_ab)
eps_ab.shape

(40725, 1, 3, 3)

In [931]:
def plot_energy_dissp(self, m, MSX):
    from scipy.integrate import cumtrapz

    sigma_ab, eps_ab = [], []
    for i in range(len(m.hist.F_t)):
        sigma_ab.append(m.fe_domain[0].xmodel.map_U_to_field(m.hist.F_t[i]))
        eps_ab.append(m.fe_domain[0].xmodel.map_U_to_field(m.hist.U_t[i]))
    sigma_ab = np.array(sigma_ab)
    eps_ab = np.array(eps_ab)

    fig_list = []

    E_damage_N_total, E_damage_T_total, E_plastic_diss_N_total, E_plastic_diss_T_total, E_iso_free_energy_total,\
    E_kin_free_energy_total, W_arr_macro_total, E_damage_diss_total, E_plastic_diss_total, W_arr_micro_elastic = 0, 0, 0, 0, 0, 0, 0, 0 , 0, 0

    # for i in range(1):

    eps_Emab_hist = eps_ab.squeeze()
    delta_eps_Emab = np.concatenate((np.zeros((3, 3))[np.newaxis, ...], np.diff(eps_Emab_hist, axis=0)), axis=0)
    eps_a = self._get_e_a(eps_Emab_hist)
    eps_a_ = np.einsum('...a->a...', eps_a)
    eps_N = eps_a_[0, ...]
    eps_T_a = np.einsum('a...->...a', eps_a_[1:, ...])
    delta_eps_N = np.concatenate((np.zeros(28, )[np.newaxis, ...], np.diff(eps_N, axis=0)), axis=0)
    delta_eps_T_a = np.concatenate((np.zeros((28, 3))[np.newaxis, ...], np.diff(eps_T_a, axis=0)), axis=0)

    omega_N, z_N, alpha_N, r_N, eps_N_p, sig_N, omega_T, z_T, alpha_T_a, eps_T_p_a, sig_T_a = \
        [], [], [], [], [], [], [], [], [], [], []
    for j in range(len(m.hist.state_vars)):
        omega_N.append(m.hist.state_vars[j][0]['omega_N'][0])
        z_N.append(m.hist.state_vars[j][0]['z_N'][0])
        alpha_N.append(m.hist.state_vars[i][0]['alpha_N'][0])
        r_N.append(m.hist.state_vars[j][0]['r_N'][0])
        eps_N_p.append(m.hist.state_vars[j][0]['eps_N_p'][0])
        sig_N.append(m.hist.state_vars[j][0]['sig_N'][0])
        omega_T.append(m.hist.state_vars[j][0]['omega_T'][0])
        z_T.append(m.hist.state_vars[j][0]['z_T'][0])
        alpha_T_a.append(m.hist.state_vars[j][0]['alpha_T_a'][0])
        eps_T_p_a.append(m.hist.state_vars[j][0]['eps_T_p_a'][0])
        sig_T_a.append(m.hist.state_vars[j][0]['sig_T_a'][0])
        

    omega_N = np.array(omega_N).squeeze()
    z_N = np.array(z_N).squeeze()
    alpha_N = np.array(alpha_N).squeeze()
    eps_N_p = np.array(eps_N_p).squeeze()
    sig_N = np.array(sig_N).squeeze()
    omega_T = np.array(omega_T).squeeze()
    z_T = np.array(z_T).squeeze()
    alpha_T_a = np.array(alpha_T_a).squeeze()
    eps_T_p_a = np.array(eps_T_p_a).squeeze()
    sig_T_a = np.array(sig_T_a).squeeze()
    eps_N_e = eps_N - eps_N_p
    eps_T_e_a = eps_T_a - eps_T_p_a
    sig_Emab_hist = MSX.NT_to_ab(sig_N, sig_T_a)

    work_microplane = np.einsum('...n,...n->...n', sig_N, delta_eps_N) + np.einsum('...na,...na->...n',
                                                                                   sig_T_a,
                                                                                   delta_eps_T_a)
    work_microplane_elastic = 0.5 * np.einsum('...n,...n->...n', sig_N, eps_N_e) + np.einsum('...na,...na->...n',
                                                                                   sig_T_a,
                                                                                   eps_T_e_a)
    W_arr_micro = cumtrapz(np.einsum('...n,...n->...', self._get_MPW(), work_microplane), initial=0)
    W_arr_micro_elastic = np.einsum('...n,...n->...', self._get_MPW(), work_microplane_elastic)
                           
    W_arr_macro = cumtrapz(np.einsum('...ij,...ij->...', sig_Emab_hist, delta_eps_Emab), initial=0)

    delta_eps_N_p = np.concatenate((np.zeros(28, )[np.newaxis, ...], np.diff(eps_N_p, axis=0)), axis=0)
    delta_eps_N_e = np.concatenate((np.zeros(28, )[np.newaxis, ...], np.diff(eps_N_e, axis=0)), axis=0)
    delta_alpha_N = np.concatenate((np.zeros(28, )[np.newaxis, ...], np.diff(alpha_N, axis=0)), axis=0)
    delta_z_N = np.concatenate((np.zeros(28, )[np.newaxis, ...], np.diff(z_N, axis=0)), axis=0)
    delta_omega_N = np.concatenate((np.zeros(28, )[np.newaxis, ...], np.diff(omega_N, axis=0)), axis=0)

    Z_N = MSX.mic_.K_N * z_N
    X_N = MSX.mic_.gamma_N * alpha_N
    Y_N = 0.5 * MSX.mic_.E_N * (eps_N - eps_N_p) ** 2.0

    plastic_work_N = np.einsum('...n,...n->...n', sig_N, delta_eps_N_p)
    elastic_work_N = np.einsum('...n,...n->...n', sig_N, delta_eps_N_e)
    kin_free_energy_N = np.einsum('...n,...n->...n', X_N, delta_alpha_N)
    iso_free_energy_N = np.einsum('...n,...n->...n', Z_N, delta_z_N)
    damage_dissip_N = np.einsum('...n,...n->...n', Y_N, delta_omega_N)

    E_plastic_work_N = cumtrapz(np.einsum('...n,...n->...', self._get_MPW(), plastic_work_N), initial=0)
    E_elastic_work_N = cumtrapz(np.einsum('...n,...n->...', self._get_MPW(), elastic_work_N), initial=0)
    E_iso_free_energy_N = cumtrapz(np.einsum('...n,...n->...', self._get_MPW(), iso_free_energy_N), initial=0)
    E_kin_free_energy_N = cumtrapz(np.einsum('...n,...n->...', self._get_MPW(), kin_free_energy_N), initial=0)
    E_plastic_diss_N = E_plastic_work_N - E_iso_free_energy_N - E_kin_free_energy_N
    E_damage_N = cumtrapz(np.einsum('...n,...n->...', self._get_MPW(), damage_dissip_N), initial=0)

    delta_eps_T_p_a = np.concatenate((np.zeros((28, 3))[np.newaxis, ...], np.diff(eps_T_p_a, axis=0)), axis=0)
    delta_eps_T_e_a = np.concatenate((np.zeros((28, 3))[np.newaxis, ...], np.diff(eps_T_e_a, axis=0)), axis=0)
    delta_alpha_T_a = np.concatenate((np.zeros((28, 3))[np.newaxis, ...], np.diff(alpha_T_a, axis=0)), axis=0)
    delta_z_T = np.concatenate((np.zeros(28, )[np.newaxis, ...], np.diff(z_T, axis=0)), axis=0)
    delta_omega_T = np.concatenate((np.zeros(28, )[np.newaxis, ...], np.diff(omega_T, axis=0)), axis=0)
    
    Z_T = MSX.mic_.K_T * z_T
    X_T = MSX.mic_.gamma_T * alpha_T_a
    Y_T = 0.5 * MSX.mic_.E_T * np.einsum('...na,...na->...n', (eps_T_a - eps_T_p_a), (eps_T_a - eps_T_p_a))

    plastic_work_T = np.einsum('...na,...na->...n', sig_T_a, delta_eps_T_p_a)
    elastic_work_T = np.einsum('...na,...na->...n', sig_T_a, delta_eps_T_e_a)
    kin_free_energy_T = np.einsum('...na,...na->...n', X_T, delta_alpha_T_a)
    iso_free_energy_T = np.einsum('...n,...n->...n', Z_T, delta_z_T)
    damage_dissip_T = np.einsum('...n,...n->...n', Y_T, delta_omega_T)

    E_plastic_work_T = cumtrapz(np.einsum('...n,...n->...', self._get_MPW(), plastic_work_T), initial=0)
    E_elastic_work_T = cumtrapz(np.einsum('...n,...n->...', self._get_MPW(), elastic_work_T), initial=0)
    E_iso_free_energy_T = cumtrapz(np.einsum('...n,...n->...',self._get_MPW(), iso_free_energy_T), initial=0)
    E_kin_free_energy_T = cumtrapz(np.einsum('...n,...n->...', self._get_MPW(), kin_free_energy_T), initial=0)
    E_plastic_diss_T = E_plastic_work_T - E_iso_free_energy_T - E_kin_free_energy_T
    E_damage_T = cumtrapz(np.einsum('...n,...n->...', self._get_MPW(), damage_dissip_T), initial=0)

    E_kin_free_energy = E_kin_free_energy_T + E_kin_free_energy_N
    E_iso_free_energy = E_iso_free_energy_T + E_iso_free_energy_N
    E_plastic_diss = E_plastic_diss_T + E_plastic_diss_N
    E_damage_diss = E_damage_T + E_damage_N
    E_plastic_work = E_plastic_work_T + E_plastic_work_N
    E_elastic_work = E_elastic_work_T + E_elastic_work_N

    E_damage_N_total += E_damage_N
    E_damage_T_total += E_damage_T
    E_plastic_diss_N_total += E_plastic_diss_N
    E_plastic_diss_T_total += E_plastic_diss_T
    E_iso_free_energy_total += E_iso_free_energy
    E_kin_free_energy_total += E_kin_free_energy
    W_arr_macro_total += W_arr_macro
    E_damage_diss_total += E_damage_diss
    E_plastic_diss_total += E_plastic_diss

    t_arr = np.linspace(0, 1, len(E_plastic_work))

    fig = plt.figure()
    ax = fig.subplots(1, 1)
    E_level = 0

    # ax2.plot(eps_Emab_hist[:, 0, 0], sig_Emab_hist[:, 0, 0])
    # ax2.plot(eps_Emab_hist[:, 0, 1], sig_Emab_hist[:, 0, 1])
    # ax2.plot(eps_Emab_hist[:, 0, 2], sig_Emab_hist[:, 0, 2])

    ax.plot(t_arr, E_damage_diss_total + E_level, color='black', lw=2)
    ax.fill_between(t_arr, E_damage_N_total + E_level, E_level, color='black',
                    hatch='|', label=r'$W$ - damage N diss');
    E_d_level = E_level + abs(E_damage_N_total)
    ax.fill_between(t_arr, abs(E_damage_T_total) + E_d_level, E_d_level, color='gray',
                    alpha=0.3, label=r'$W$ - damage T diss');

    E_level = abs(E_damage_diss_total)

    ax.plot(t_arr, E_plastic_diss_total + E_level, lw=1., color='red')
    ax.fill_between(t_arr, E_plastic_diss_N_total + E_level, E_level, color='red',
                    hatch='-', label=r'$W$ - plastic N diss')
    E_d_level = E_level + E_plastic_diss_N_total
    ax.fill_between(t_arr, E_plastic_diss_T_total + E_d_level, E_d_level, color='red',
                    alpha=0.3, label=r'$W$ - plastic T diss')
    E_level += E_plastic_diss_total

    ax.plot(t_arr, abs(E_iso_free_energy_total) + E_level, '-.', lw=0.5, color='black')
    ax.fill_between(t_arr, abs(E_iso_free_energy_total) + E_level, E_level, color='royalblue',
                    hatch='|', label=r'$W$ - iso free energy')

    E_level += abs(E_iso_free_energy_total)
    ax.plot(t_arr, abs(E_kin_free_energy_total) + E_level, '-.', color='black', lw=0.5)
    ax.fill_between(t_arr, abs(E_kin_free_energy_total) + E_level, E_level, color='royalblue', alpha=0.2,
                    label=r'$W$ - kin free energyy')

    E_level += abs(E_kin_free_energy_total)

    ax.plot(t_arr, W_arr_micro_elastic + E_level, lw=0.5, color='black', label=r'$W$ - Input work')
    # ax.plot(t_arr, G_arr, '--', color='black', lw = 0.5, label=r'$W^\mathrm{inel}$ - Inelastic work')
    ax.fill_between(t_arr, W_arr_micro_elastic + E_level, E_level, color='green', alpha=0.2, label=r'$W$ - stored energy')
    ax.set_xlabel('$t$ [-]');
    ax.set_ylabel(r'$E$ [Nmm]')
    ax.legend()
    fig_list.append(fig)

    return fig_list

In [936]:
for m in m_list_monotonic:
    energydissipation = EnergyDissipation()
    fig = plot_energy_dissp(energydissipation, m, mpl)

[[ 1.71540826e-03 -8.57704014e-04 -8.57704014e-04]
 [-1.72122261e-03  8.60611606e-04 -8.60611606e-04]
 [-1.72122261e-03 -8.60611606e-04  8.60611606e-04]
 [ 1.71540826e-03  8.57704014e-04  8.57704014e-04]
 [ 6.12762021e-04 -1.14342945e-03 -1.14342945e-03]
 [-6.13659431e-04  1.14510436e-03 -1.14510436e-03]
 [-6.13659431e-04 -1.14510436e-03  1.14510436e-03]
 [ 6.12762021e-04  1.14342945e-03  1.14342945e-03]
 [ 3.70530782e-04  5.94340250e-04 -2.58863885e-03]
 [-3.72958239e-04 -5.94025479e-04 -2.58989194e-03]
 [-3.72958239e-04  5.94025479e-04  2.58989194e-03]
 [ 3.70530782e-04 -5.94340250e-04  2.58863885e-03]
 [ 3.70530782e-04 -2.58863885e-03  5.94340250e-04]
 [-3.72958239e-04  2.58989194e-03  5.94025479e-04]
 [-3.72958239e-04 -2.58989194e-03 -5.94025479e-04]
 [ 3.70530782e-04  2.58863885e-03 -5.94340250e-04]
 [ 1.21312426e-03 -1.62527873e-04 -1.62527873e-04]
 [-1.22789037e-03  1.64505569e-04 -1.64505569e-04]
 [-1.22789037e-03 -1.64505569e-04  1.64505569e-04]
 [ 1.21312426e-03  1.62527873e-

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[ 8.30297405e-04 -4.15149038e-04 -4.15148492e-04]
 [-1.69098114e-03  8.45490648e-04 -8.45490646e-04]
 [-1.69098106e-03 -8.45490609e-04  8.45490607e-04]
 [ 8.30297405e-04  4.15148489e-04  4.15149041e-04]
 [ 2.53286988e-04 -4.72640457e-04 -4.72639569e-04]
 [-5.69114307e-04  1.06198185e-03 -1.06198185e-03]
 [-5.69114362e-04 -1.06198195e-03  1.06198195e-03]
 [ 2.53286988e-04  4.72639568e-04  4.72640458e-04]
 [-9.05010739e-05  5.33184062e-04 -1.89936910e-03]
 [-6.81484360e-04 -3.69951244e-04 -2.06216141e-03]
 [-6.81483496e-04  3.69951533e-04  2.06216163e-03]
 [-9.05005156e-05 -5.33183824e-04  1.89936877e-03]
 [-9.05005153e-05 -1.89936878e-03  5.33183825e-04]
 [-6.81484359e-04  2.06216141e-03  3.69951245e-04]
 [-6.81483495e-04 -2.06216163e-03 -3.69951534e-04]
 [-9.05010735e-05  1.89936911e-03 -5.33184063e-04]
 [ 3.87656610e-04 -5.19361266e-05 -5.19359529e-05]
 [-1.21164952e-03  1.62330020e-04 -1.62330017e-04]
 [-1.21164912e-03 -1.62329966e-04  1.62329963e-04]
 [ 3.87656610e-04  5.19359496e-

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[ 2.90494344e-04 -1.45247232e-04 -1.45247224e-04]
 [-1.59190948e-03  7.95954804e-04 -7.95954804e-04]
 [-1.59190948e-03 -7.95954804e-04  7.95954804e-04]
 [ 2.90494344e-04  1.45247224e-04  1.45247232e-04]
 [ 8.86169633e-05 -1.65361553e-04 -1.65361541e-04]
 [-7.27638182e-04  1.35779145e-03 -1.35779145e-03]
 [-7.27638174e-04 -1.35779143e-03  1.35779143e-03]
 [ 8.86169633e-05  1.65361541e-04  1.65361553e-04]
 [-1.72325885e-04  4.88194964e-04 -1.64964268e-03]
 [-7.00655748e-04 -3.03670857e-04 -1.83397100e-03]
 [-7.00655736e-04  3.03670861e-04  1.83397100e-03]
 [-1.72325878e-04 -4.88194962e-04  1.64964268e-03]
 [-1.72325878e-04 -1.64964268e-03  4.88194962e-04]
 [-7.00655748e-04  1.83397100e-03  3.03670857e-04]
 [-7.00655736e-04 -1.83397100e-03 -3.03670861e-04]
 [-1.72325885e-04  1.64964268e-03 -4.88194964e-04]
 [ 1.35628592e-04 -1.81707211e-05 -1.81707186e-05]
 [-1.14790314e-03  1.53789650e-04 -1.53789650e-04]
 [-1.14790313e-03 -1.53789649e-04  1.53789649e-04]
 [ 1.35628592e-04  1.81707186e-

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2D 1 single mp

In [3]:
m_inelastic = MS12D(**CAxisymm)

In [4]:
tf_precrompression = TFBilinear(loading_ratio = 1.0, time_ratio=0.1)
tf_sliding = TFBilinear(loading_ratio = 0.0, time_ratio=0.1)

In [5]:
xmodel = XDomainSinglePoint2D()
m = TStepBC(
    domains=[(xmodel, m_inelastic),],
    bc=[BCDof(
            var='u', dof=2, value=-0.01,
         time_function=tf_sliding) , BCDof(
            var='f', dof=0, value=-5,
         time_function=tf_precrompression)]
)

In [ ]:
m.sim.tline.trait_set(step=0.1)
m.sim.reset()
m.sim.run()